In [19]:
import itertools
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=["L", "T", "mu"])
y = []
mersenne = lambda L, T, mu: (np.sqrt(T/mu) / (2*L))# * (1 + np.random.randn()*0.1)

Ls = np.logspace(0, 4, num=12, base=2, endpoint=False)
Ts = np.logspace(0, 2, num=5, base=2, endpoint=False)
mus = np.logspace(0, 2, num=5, base=2, endpoint=False)

for (L, T, mu) in itertools.product(Ls, Ts, mus):
    y.append(mersenne(L, T, mu))
    df = df.append({"L": L, "T": T, "mu": mu}, ignore_index=True)
y = np.array(y)

In [4]:
import torch
import numpy as np
import sympy as sp
import os, sys
import symbolicregression
import requests
from IPython.display import display

model_path = "model.pt" 
try:
    if not os.path.isfile(model_path): 
        url = "https://dl.fbaipublicfiles.com/symbolicregression/model1.pt"
        r = requests.get(url, allow_redirects=True)
        open(model_path, 'wb').write(r.content)
    if not torch.cuda.is_available():
        model = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        model = torch.load(model_path)
        model = model.cuda()
    print(model.device)
    print("Model successfully loaded!")

except Exception as e:
    print("ERROR: model not loaded! path was: {}".format(model_path))
    print(e)

est = symbolicregression.model.SymbolicTransformerRegressor(
    model=model,
    max_input_points=200,
    n_trees_to_refine=100,
    rescale=True
    )    

cuda:0
Model successfully loaded!


In [13]:
x = df.to_numpy()

est.fit(x,y)
replace_ops = {"add": "+", "mul": "*", "sub": "-", "pow": "**", "inv": "1/"}
model_str = est.retrieve_tree(with_infos=True)["relabed_predicted_tree"].infix()
for op,replace_op in replace_ops.items():
    model_str = model_str.replace(op,replace_op)
display(sp.parse_expr(model_str))

0.0033399999999999997 + 0.883/((0.393 + 8.94/(0.079272033982009744*x_2 - (0.0128 - 6.93*Abs((0.023531034256704861*x_0 - 0.15796433521939713)*(1.3846643490307378*x_1 - 9.3702442550130565)*(0.34062742986156153*x_2 + 7.0703399132667879)))*(0.066325422318572348*x_1 + 0.056448300184874624) - 0.14384398359949752))*(0.27683569713770423*x_0 + 0.5486548797717987)*(0.53171111002780333*x_2 + 0.40150620607498618))

(300,)